In [ ]:
import pandas as pd
from glob import glob
from simpledbf import Dbf5
import os
import pyproj
import numpy as np

In [ ]:
files = glob('C:\\Users\\jlandman\\Desktop\\newData\\CL_Schaefer\\Schaefer_rawdata-chile\\DD\\*.dbf')
files

# Echaurren and Plomo have no GENERAL entry by M. Schaefer

In [ ]:
pop_list = ['del_potro','tronquitos','apado','juncal_n','rio_blanco','tupungatito','marmolejo', 'echaurren', 'terrestre'] 

In [ ]:
for i in files:
    print(i)
    for j in pop_list:
        if j in i:
            files.remove(i)
files

In [ ]:
thin_dict = {'bello':11,
             'callaqui':21,
             'cipreses_norte':43,
             'del_rincon':4,
             'echaurren':1,
             'esmeralda':15,
             'la_paloma':7,
             'llaima':16,
             'meson_alto':11,
             'mocho':28,
             'mocho_terrestre':4,
             'osorno':37,
             'plomo':9,
             'san_francisco':4,
             'sollipulli':9,
             's_velluda':32,
             'tinguiririca_1':16,
             'tinguiririca_3':12,
             'tinguiririca_4':9,
             'tronador':18,
             'universidad':20,
             'villarrica':44,
             'villarrica_terrestre':5,
             'yeso':9,
             'yeso_2':4
             }

name_dict = {'bello':'Bello',
             'callaqui':'Callaqui',
             'cipreses_norte':'Cipreses Norte',
             'del_rincon':'Del Rincon',
             'echaurren':'Echaurren',
             'esmeralda':'Esmeralda',
             'la_paloma':'La Paloma',
             'llaima':'Llaima',
             'meson_alto':'Meson Alto',
             'mocho':'Mocho-Chosh',
             'mocho_terrestre':'Mocho-Chosh',
             'osorno':'Osorno',
             'plomo':'Cerro el Plomo',
             'san_francisco':'San Francisco',
             'sollipulli':'Sollipulli',
             's_velluda':'Velluda',
             'tinguiririca_1':'Tinguiririca 1',
             'tinguiririca_3':'Tinguiririca 3',
             'tinguiririca_4':'Tinguiririca 4',
             'tronador':'Tronador',
             'universidad':'Universidad',
             'villarrica':'Villarrica',
             'villarrica_terrestre':'Villarrica',
             'yeso':'Yeso',
             'yeso_2':'Yeso 2'
             }

In [ ]:
# read the T table
general = pd.read_excel('C:\\Users\\jlandman\\Desktop\\newData\\CL_Schaefer\\GlaThiDa2-0_MS_Johannes.xls')

In [ ]:
for file in files:
    new_data = pd.DataFrame([])
    fname = os.path.basename(file)
    dbf = Dbf5(file)
    data = dbf.to_dataframe()
    print(fname)
    data = data.groupby(data.index // thin_dict[fname.split('.')[0]]).mean() # average over every x values as specified above
    
    print(data.columns.values)
    
    # ice thickness
    try:
        new_data['THICKNESS'] = data['esp'].map('{:.1f}'.format)
    except KeyError:
        try:
            new_data['THICKNESS'] = data['Espesor'].map('{:.1f}'.format)
        except KeyError:
            new_data['THICKNESS'] = data['Esp'].map('{:.1f}'.format)
            
    # lat
    try:
        new_data['POINT_LAT'] = data['y'].map('{:.4f}'.format)
    except KeyError:
        try:
            new_data['POINT_LAT'] = data['Norte'].map('{:.4f}'.format)
        except KeyError:
            new_data['POINT_LAT'] = data['N'].map('{:.4f}'.format)
        
    # lon
    try:
        new_data['POINT_LON'] = data['x'].map('{:.4f}'.format)
    except KeyError:
        try:
            new_data['POINT_LON'] = data['Este'].map('{:.4f}'.format)
        except KeyError:
            new_data['POINT_LON'] = data['E'].map('{:.4f}'.format)
    
    new_data['GLACIER_NAME'] = general[general.GLACIER_NAME == name_dict[os.path.split(file)[1].split('.')[0]]].GLACIER_NAME.iloc[0]
    new_data['GlaThiDa_ID'] = general[general.GLACIER_NAME == name_dict[os.path.split(file)[1].split('.')[0]]].GlaThiDa_ID.iloc[0]
    new_data['POLITICAL_UNIT'] = general[general.GLACIER_NAME == name_dict[os.path.split(file)[1].split('.')[0]]].POLITICAL_UNIT.iloc[0]
    new_data['SURVEY_DATE'] = general[general.GLACIER_NAME == name_dict[os.path.split(file)[1].split('.')[0]]].SURVEY_DATE.iloc[0]
    new_data['POINT_ID'] = range(1, len(new_data)+1)
    new_data['ELEVATION'] = np.nan
    new_data['THICKNESS_UNCERTAINTY'] = np.nan
    new_data['DATA_FLAG'] = np.nan
    new_data['REMARKS'] = ''
    
    new_data = new_data[['GlaThiDa_ID','POLITICAL_UNIT','GLACIER_NAME','SURVEY_DATE','POINT_ID','POINT_LAT','POINT_LON',
                         'ELEVATION','THICKNESS','THICKNESS_UNCERTAINTY','DATA_FLAG','REMARKS']]

    new_data.to_excel(os.path.join(os.path.dirname(file),fname.split('.')[0]+'.xls'), index=False)